In [1]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_validation import train_test_split

import pandas as pd
from urllib2 import Request, urlopen
from json import dumps, loads

from features import feature_extraction

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
api_url = 'http://api.parkwhiz.com/v4/quotes?q=coordinates:41.8857256,-87.6369590'
api_key = '&api_key=62d882d8cfe5680004fa849286b6ce20'
times = [
    '&start_time=2017-04-14T15:00&end_time=2017-04-14T18:00',
    '&start_time=2017-05-14T15:00&end_time=2017-05-14T18:00',
    '&start_time=2017-06-14T15:00&end_time=2017-06-14T18:00',
    '&start_time=2017-07-14T15:00&end_time=2017-07-14T18:00',
    '&start_time=2017-08-14T15:00&end_time=2017-08-14T18:00',
    '&start_time=2017-09-14T15:00&end_time=2017-09-14T18:00',
    '&start_time=2017-12-23T15:00&end_time=2017-12-23T18:00',
]

frames = []

for time in times:    
    req = Request(api_url + api_key + time)
    page = urlopen(req)
    content = page.read()
    json_data = loads(content)
    frames.append(feature_extraction(json_data))
    
frame = pd.concat(frames)
frame = frame.drop_duplicates('id')
frame.head()

,id,name,lat,lon,coordinates,address,city,seller_id,price,accessible,valet,security,restrooms,printed_pass,reentry_allowed
0,8343,150 N. Wacker Dr. - Valet,41.8848,-87.6372,"[41.8848377337, -87.6372286677]",150 N. Upper Wacker Dr.,Chicago,624,12,0,1,0,0,0,0
1,6460,Franklin & Lake Self Park,41.8852,-87.6356,"[41.885230118, -87.6355737448]",180 N. Franklin St.,Chicago,652,15,1,0,1,0,0,0
2,3871,225 W. Wacker Dr. Building,41.8859,-87.6353,"[41.8859381122, -87.6352616959]",281 N. Franklin St.,Chicago,602,10,1,0,0,0,0,0
3,9529,319 W. Randolph - Lot,41.8843,-87.636,"[41.8842716176, -87.6360226795]",319 W. Randolph St.,Chicago,652,30,1,0,0,0,0,0
4,3186,165 N. Canal St. - Valet Kiosk,41.8851,-87.6397,"[41.885097127, -87.6396544576]",165 N. Canal St.,Chicago,211,10,1,1,0,0,0,0


In [8]:
# Re-Entry Allowed and printed pass are not that relevant, only 1 row is true so we exclude
df_train = frame[['lat', 'lon', 'accessible', 'valet', 'security', 'restrooms']]

In [12]:
df_train.head()

,lat,lon,accessible,valet,security,restrooms
0,41.8848,-87.6372,0,1,0,0
1,41.8852,-87.6356,1,0,1,0
2,41.8859,-87.6353,1,0,0,0
3,41.8843,-87.636,1,0,0,0
4,41.8851,-87.6397,1,1,0,0


In [48]:
# Ensemble. One for lat lon with haversine distance and the other with a categorical jaccard metric
# X_d_train = distance knnR data
# X_c_train = categorical knnR data
X_d_train, X_d_test, y_d_train, y_d_test = train_test_split(df_train[['lat', 'lon']],frame['price'],test_size=0.2,random_state=574)
X_c_train, X_c_test, y_c_train, y_c_test = train_test_split(df_train[['accessible', 'valet', 'security', 'restrooms']],frame['price'],test_size=0.2,random_state=574)

In [49]:
knnRDistance = KNeighborsRegressor(n_neighbors=3, metric='haversine')
knnRCategorical = KNeighborsRegressor(n_neighbors=3, metric='jaccard')

In [50]:
knnRDistance.fit(X_d_train,y_d_train)
knnRCategorical.fit(X_c_train,y_c_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='jaccard',
          metric_params=None, n_jobs=1, n_neighbors=3, p=2,
          weights='uniform')

In [51]:
from sklearn.metrics import mean_squared_error
y_d_pred = knnRDistance.predict(X_d_test)
y_c_pred = knnRCategorical.predict(X_c_test)

print mean_squared_error(y_d_pred, y_d_test)
print mean_squared_error(y_c_pred, y_c_test)

24.1666666667
55.8240740741


In [27]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
from repository.mlrepository import MetaProps, MetaNames

ImportError: No module named repository.mlrepositoryclient

In [ ]:
wml_credentials = {
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "5eJV3UHu5LQEVQObnm8ciAywGzvSm7y47quRrlI7Luq3lOzm9fi0i9yQkv70gO54HxGxQ3pIogjgEOjN0TGDTcL0h32gVzPkwMbmHXNpi+FQYUqQmv73SQJrb1WXWeZv",
  "username": "33e97d56-995f-4f77-b0f1-646731e50702",
  "password": "3c2a4a2d-7020-46c5-a9c8-954122ac9d60",
  "instance_id": "00195eb4-c665-4384-9e21-57e59ac66f5f"
}

In [ ]:
ml_repository_client = MLRepositoryClient(wml_credentials['url'])
ml_repository_client.authorize(wml_credentials['username'], wml_credentials['password'])

In [ ]:
props = MetaProps({MetaNames.AUTHOR_NAME:"Aleksandar Velkoski", MetaNames.AUTHOR_EMAIL:"avelkoski@ethventures.io"})
model_artifact = MLRepositoryArtifact(knnR, name="SpotExchange KNeighborsRegressor", meta_props=props)

In [ ]:
saved_model = ml_repository_client.models.save(model_artifact)

In [ ]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{}/v3/identity/token'.format(wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

In [ ]:
endpoint_instance = wml_credentials['url'] + "/v3/wml_instances/" + wml_credentials['instance_id']
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken} 

response_get_instance = requests.get(endpoint_instance, headers=header)

In [ ]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')

In [ ]:
print endpoint_published_models

In [ ]:
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
response_get = requests.get(endpoint_published_models, headers=header)

In [ ]:
print mltoken

In [ ]:
[endpoint_deployments] = [x.get('entity').get('deployments').get('url') for x in json.loads(response_get.text).get('resources') if x.get('metadata').get('guid') == saved_model.uid]

In [ ]:
payload_online = {"name": "knnR-SpotExchange", "description": "KNearestNeighbor Regressor for SpotExchange Price Predictions", "type": "online"}
response_online = requests.post(endpoint_deployments, json=payload_online, headers=header)

In [ ]:
scoring_url = json.loads(response_online.text).get('entity').get('scoring_url')

print(scoring_url)

In [ ]:
values = [[41.88523011803571, -87.63557374477386],
       [41.88593811224804, -87.6352616958502],
       [41.88427161764708, -87.63602267950773]]

In [ ]:
payload_scoring = {"values": values}

In [ ]:
response_scoring = requests.post(scoring_url, json=payload_scoring, headers=header)

In [ ]:
print response_scoring.text